In [17]:
import sagemaker
from sagemaker.local import LocalSession

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/pytorch-health"

role = sagemaker.get_execution_role()

In [18]:
import torchvision
from torchvision import datasets, transforms
from packaging.version import Version

In [19]:
# inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
# print('input spec (in this case, just an S3 path): {}'.format(inputs))

In [20]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='food101.py',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.m5.xlarge',
                    hyperparameters={
                        'epochs': 1,
                        'backend': 'gloo'
                    })

In [21]:
data_uri = "s3://sagemaker-us-east-1-119266240050/sagemaker/pytorch-health"

In [ ]:
estimator.fit({'training': data_uri})

2022-05-10 18:37:22 Starting - Starting the training job...
2022-05-10 18:37:46 Starting - Preparing the instances for trainingProfilerReport-1652207842: InProgress
.........
2022-05-10 18:39:14 Downloading - Downloading input data...........................................................................
2022-05-10 18:51:57 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-10 18:51:59,477 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-10 18:51:59,480 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-05-10 18:51:59,492 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-10 18:51:59,500 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-10 18:52:00,028 sagemaker-training-toolkit INFO   

In [23]:
print("end")

end


In [24]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------!

In [18]:
def _get_train_data_loader(batch_size, training_dir, is_distributed, **kwargs):
#     logger.info("Get train data loader")
    dataset = food101_custom(
        training_dir,
        'train',
    )
    train_sampler = (
        torch.utils.data.distributed.DistributedSampler(dataset) if is_distributed else None
    )
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=train_sampler is None,
        sampler=train_sampler,
        **kwargs
    )

In [17]:
from torch.utils.data import Dataset
class food101_custom(Dataset):
    def __init__(self, path, partition, transform=None, target_transform=None):
        
        self.path = path.split("data")[0]
        print(self.path)
        
        with open(f'{path}/data/food-101/meta/{partition}.json', 'r') as f:
            self.data = json.load(f)
        
        self.name2label = {name:i for i, name in enumerate(self.data.keys())}
        self.label2name = {v:k for k,v in self.name2label.items()}
        
        self.data_list_form = []
        for labelname, image_list in self.data.items():
            for image_url in image_list:
                self.data_list_form.append((image_url, labelname))

        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.data_list_form)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.path,'food-101','images', self.data_list_form[idx][0]+'.jpg')
        image = Image.open(img_path)
        image = transforms.ToTensor()(image)
        image = transforms.Resize((224,224))(image)
        
        label = self.name2label[self.data_list_form[idx][1]]
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    

In [8]:
import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset
from torchvision import datasets, transforms

In [16]:
with open("data/food-101/meta/train.json", 'r') as f:
    print(f)

<_io.TextIOWrapper name='data/food-101/meta/train.json' mode='r' encoding='UTF-8'>


In [6]:
from torchvision import datasets

In [10]:
datasets.MNIST(
    'data',
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [17]:
from os import walk

a = []
for (dirpath, dirnames, filenames) in walk('data/food-101/images/'):
    folders = dirnames
    break


In [16]:
import os
import json
from PIL import Image


In [55]:
import argparse
import json
import logging
import os
import sys

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset
from torchvision import datasets, transforms

class food101_custom(Dataset):
    def __init__(self, path, partition, transform=None, target_transform=None):
        
#         logger.info("====================================================================================")
#         logger.info(f"path: {path}")
#         os.system("pwd")
#         logger.info("====================================================================================")
        
        
        self.path = path
        
        with open(os.path.join(self.path,'food-101','meta',f'{partition}.json'), 'r') as f:
            self.data = json.load(f)
        
        self.name2label = {name:i for i, name in enumerate(self.data.keys())}
        self.label2name = {v:k for k,v in self.name2label.items()}
        
        self.data_list_form = []
        for labelname, image_list in self.data.items():
            for image_url in image_list:
                self.data_list_form.append((image_url, labelname))

        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.data_list_form)

    def __getitem__(self, idx):
        
        success = False
        
        while not success:
            
            try:
                
                preprocess = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])
                
                img_path = os.path.join(self.path,'food-101','images', self.data_list_form[idx][0]+'.jpg')
                image = Image.open(img_path)
                
                preprocess = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ])
                
                image = preprocess(image)
            
                if tuple(image.shape) != (3, 224, 224):
                    print(tuple(image.shape))
                    print(self.data_list_form[idx])
                    print()
                    idx = (idx+1) % len(self.data_list_form)
                    continue
                
                success = True
                
            except:
                idx = (idx+1) % len(self.data_list_form)
        
        label = self.name2label[self.data_list_form[idx][1]]
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
def _get_train_data_loader(batch_size, training_dir, is_distributed, **kwargs):
#     logger.info("Get train data loader")
    dataset = food101_custom(
        training_dir,
        'train',
    )
    train_sampler = (
        torch.utils.data.distributed.DistributedSampler(dataset) if is_distributed else None
    )
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=train_sampler is None,
        sampler=train_sampler,
        **kwargs
    )

In [56]:
data_loader = _get_train_data_loader(512, 'data', False)

In [57]:
for X, y in data_loader:
    print(X.shape)
    break

torch.Size([512, 3, 224, 224])


In [ ]:
tuple(X.shape)

In [28]:
from PIL import Image
import requests
import numpy as np

In [164]:
url = "https://s3-media2.fl.yelpcdn.com/bphoto/dxMzmO9yAx5uWwiqzwjXCw/o.jpg"
im = Image.open(requests.get(url, stream=True).raw).resize((224,224))
im = np.array(im).astype(np.float32)
im = np.moveaxis(im, 2, 0)
im = np.expand_dims(im, axis=0)



In [150]:
im = im/2

In [151]:
im.shape

(1, 3, 224, 224)

In [165]:
response = predictor.predict(im)

In [166]:
response.shape

(1, 101)

array([-8.64702606e+00, -2.66228504e+01, -1.70597916e+01, -1.32028160e+01,
       -3.99878464e+01, -4.67819214e+00, -5.31282043e+00, -1.46925697e+01,
       -3.82949829e+01, -1.78865509e+01, -2.18997231e+01, -1.54518661e+01,
       -2.49958534e+01, -2.10971832e+01, -2.78014679e+01, -1.98006115e+01,
       -1.59833908e+01, -2.42354546e+01, -1.43897285e+01, -4.39290161e+01,
       -1.19977341e+01, -2.95379829e+01, -3.38007088e+01, -1.89820658e-02,
       -5.42748260e+00, -2.44128914e+01, -5.80835571e+01, -6.77180099e+01,
       -5.48009491e+01, -5.58306503e+01, -6.47310638e+01, -5.68427544e+01,
       -5.84883347e+01, -5.93338356e+01, -6.06045723e+01, -5.44488602e+01,
       -5.07026215e+01, -5.99098625e+01, -5.06388702e+01, -5.08390732e+01,
       -6.17877350e+01, -6.49610138e+01, -5.49770470e+01, -4.98209496e+01,
       -5.58120079e+01, -5.92909241e+01, -5.13771286e+01, -6.18253517e+01,
       -6.14178581e+01, -5.66344452e+01, -5.26415863e+01, -6.90209961e+01,
       -5.67531853e+01, -

In [157]:
from scipy.special import softmax

In [159]:
np.argmax(softmax(response[0]))

23

In [160]:
softmax(response[0])[23]

0.98119704262086